In [ ]:
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import preprocessing
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import pandas as pd
import phate
import math
import random
import gc
import scprep
from datetime import datetime, time
from matplotlib.animation import ImageMagickWriter
import matplotlib.animation as animation
import zipfile
from urllib.request import urlopen
import scipy.stats as st
from scipy.stats import norm
from scipy.stats import gaussian_kde
from scipy.stats import kde
from scipy.stats import binned_statistic
from scipy.stats import f_oneway
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams['pdf.fonttype'] = 42
print(sns.__version__)
from anndata import AnnData
import scanpy as sc
from delve import *
import anndata as ad
from sklearn.preprocessing import MinMaxScaler
from kh import sketch
from sklearn.cluster import KMeans
import umap
print(sc.__version__)
today = datetime.now().strftime("%m%d%Y-%H%M")

In [ ]:
#Read back in the subsampled adata file
adata_save_path = r'your/filepath/here.h5ad'
standard_trimmed_noPSTAT5_adata_sub = anndata.read_h5ad(adata_save_path)

In [4]:
def laplacian_score_fs(adata = None,
                    k: int  = None,
                    n_jobs: int  = -1):

    X, feature_names, obs_names = parse_input(adata)
    W = construct_affinity(X = X, k = k, n_jobs = n_jobs)
    scores = laplacian_score(X = X, W = W)
    predicted_features = pd.DataFrame(scores, index = feature_names, columns = ['laplacian_score'])
    predicted_features = predicted_features.sort_values(by = 'laplacian_score', ascending = True)

    return predicted_features 

In [5]:
l_score_standard = laplacian_score_fs(standard_trimmed_noPSTAT5_adata_sub, k = 100)

In [ ]:
len(l_score_standard)

In [ ]:
l_score_standard.index[:46]

Code below taking inputs from Slingshot processing (Done in R) and any provided Clustering strategy.
Phate must be run and the PHATE coordinates saved using methods from the PHATE plotting notebook.
Produces a graph that plots specified clusters and overlays the Lineage information provided by Slingshot.

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from PIL import Image

# Create a folder to save the images if it does not exist
output_folder = 'output_plots'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Step 1: Retrieve the PHATE embeddings from the AnnData object
embedding = standard_trimmed_noPSTAT5_adata_sub.obsm['X_phate']

# Load CSV data
csv_path = r'your/trajectories/here.csv'
lineage_data = pd.read_csv(csv_path)  # Changed variable name to lineage_data

# Identify unique lineages and assign a color to each
unique_lineages = lineage_data['Lineage'].unique()
lineage_color_map = plt.get_cmap('tab10', len(unique_lineages))
lineage_colors = lineage_color_map(np.linspace(0, 1, len(unique_lineages)))
lineage_color_dict = dict(zip(unique_lineages, lineage_colors))

for feature in standard_trimmed_noPSTAT5_adata_sub.var_names:
    feature_intensity = standard_trimmed_noPSTAT5_adata_sub[:, feature].X.flatten()
    
    # Limit the visualized range from the 1st to the 99th percentile
    vmin, vmax = np.percentile(feature_intensity, [1, 99])
    
    views = [(40, 40), (40, 120), (40, 210), (40, 300)]
    
    for i, (elev, azim) in enumerate(views, start=1):
        # Create a 3D subplot for points with white background
        fig_points = plt.figure(figsize=(20, 16))
        ax_points = fig_points.add_subplot(1, 1, 1, projection='3d', facecolor='white')
        scatter = ax_points.scatter(
            embedding[:, 0], embedding[:, 1], embedding[:, 2], 
            c=feature_intensity, cmap='viridis', vmin=vmin, vmax=vmax, s=50, zorder=1
        )
        
        ax_points.view_init(elev=elev, azim=azim)
        ax_points.set_xlabel('PHATE 1')
        ax_points.set_ylabel('PHATE 2')
        ax_points.set_zlabel('PHATE 3')
        ax_points.set_title(f'{feature} - View {i} - Elev {elev}, Azim {azim}')
        ax_points.set_facecolor('white')  # Set the background color to white

        # Capture the axis limits
        xlim = ax_points.get_xlim()
        ylim = ax_points.get_ylim()
        zlim = ax_points.get_zlim()
        
        points_file = os.path.join(output_folder, f'{feature}_points_view_{i}.png')
        fig_points.savefig(points_file, transparent=True)
        plt.close(fig_points)
        
        # Create a 3D subplot for lines with the same axis limits and transparent background
        fig_lines = plt.figure(figsize=(20, 16))
        ax_lines = fig_lines.add_subplot(1, 1, 1, projection='3d')
        fig_lines.patch.set_alpha(0)  # Set the figure background to be transparent
        ax_lines.patch.set_alpha(0)  # Set the axes background to be transparent
        ax_lines.axis('off')  # Remove the axis
        lineage_handles = []
        for lineage, lineage_color in lineage_color_dict.items():
            lineage_subset = lineage_data[lineage_data['Lineage'] == lineage].sort_values('Index')
            # Apply cutoff for Lineage 2
            if (lineage == 2) and ('Index' in lineage_subset.columns):
                lineage_subset = lineage_subset[lineage_subset['Index'] <= 75]
            line, = ax_lines.plot(lineage_subset['PHATE_1'], lineage_subset['PHATE_2'], lineage_subset['PHATE_3'], 
                                  color=lineage_color, label=f'Lineage {lineage}', linewidth=5, zorder=2)
            lineage_handles.append(line)
        
        ax_lines.view_init(elev=elev, azim=azim)
        ax_lines.set_xlim(xlim)
        ax_lines.set_ylim(ylim)
        ax_lines.set_zlim(zlim)
        # Remove axis labels and titles
        ax_lines.set_xlabel('')
        ax_lines.set_ylabel('')
        ax_lines.set_zlabel('')
        ax_lines.set_title('')
        lines_file = os.path.join(output_folder, f'{feature}_lines_view_{i}.png')
        fig_lines.savefig(lines_file, transparent=True)
        plt.close(fig_lines)
        
        # Combine images
        points_image = Image.open(points_file)
        lines_image = Image.open(lines_file)
        combined_image = Image.alpha_composite(points_image.convert('RGBA'), lines_image.convert('RGBA'))
        combined_file = os.path.join(output_folder, f'{feature}_combined_view_{i}.png')
        combined_image.save(combined_file)
        
        # Save the legend as a separate image
        fig_legend = plt.figure(figsize=(10, 5))
        ax_legend = fig_legend.add_subplot(111)
        ax_legend.axis('off')  # Hide the axes for the legend
        legend_handles = lineage_handles
        legend_labels = [handle.get_label() for handle in legend_handles]
        legend = ax_legend.legend(handles=legend_handles, labels=legend_labels, loc='center', frameon=False)
        legend_file = os.path.join(output_folder, f'{feature}_legend_view_{i}.png')
        fig_legend.savefig(legend_file, transparent=True, bbox_inches='tight', pad_inches=0)
        plt.close(fig_legend)
